# 2.5. Automatic Differentiation¶

autograd package is part of pytorch


In [42]:
from torch import tensor
import torch

"""
Assume we want to autograd function `y = 2x^Tx`
"""
x : tensor = torch.arange(4.0, requires_grad=True)
x


tensor([0., 1., 2., 3.], requires_grad=True)

In [43]:
x.grad # The gradient is None by default


In [44]:
y : tensor = 2 * torch.dot(x, x)
y


tensor(28., grad_fn=<MulBackward0>)

In [45]:
y.backward()
x.grad


tensor([ 0.,  4.,  8., 12.])

In [46]:
x.grad == 4 * x


tensor([True, True, True, True])

In [47]:
x.grad.zero_() # Reset gradient
y = x.sum()
y.backward()
x.grad


tensor([1., 1., 1., 1.])

## Backward for non-scalar variables


In [48]:
x, x.grad


(tensor([0., 1., 2., 3.], requires_grad=True), tensor([1., 1., 1., 1.]))

In [49]:
x.grad.zero_()
x, x.grad


(tensor([0., 1., 2., 3.], requires_grad=True), tensor([0., 0., 0., 0.]))

In [50]:
y = x * x
y.backward(gradient=torch.ones(len(y)))
x.grad


tensor([0., 2., 4., 6.])

## 2.5.3 Detaching computation

[http://d2l.ai/chapter_preliminaries/autograd.html#detaching-computation]()


In [51]:
x.grad.zero_()
x, x.grad


(tensor([0., 1., 2., 3.], requires_grad=True), tensor([0., 0., 0., 0.]))

In [52]:
y = x * x
u = y.detach()
z = u * x
u, z

(tensor([0., 1., 4., 9.]),
 tensor([ 0.,  1.,  8., 27.], grad_fn=<MulBackward0>))

In [53]:
z.sum().backward()
x.grad, x.grad == u


(tensor([0., 1., 4., 9.]), tensor([True, True, True, True]))

In [54]:
"""
Note that while this procedure detaches y’s ancestors from the graph leading to z, the computational graph leading to y persists and thus we can calculate the gradient of y with respect to x.
"""

x.grad.zero_()
x.grad


tensor([0., 0., 0., 0.])

In [55]:
y.sum().backward() # this is one-time operation! Once y is computed you can call .backward() only once!
x.grad, x.grad == 2 * x


(tensor([0., 2., 4., 6.]), tensor([True, True, True, True]))

## 2.5.4 Gradients and Python control flow

Automatic differentiation can still calculate the gradient of the resulting variable even if function involves complicated computations with branching, loops etc.

In [56]:
def f(a):
    b = a * 2
    while b.norm() < 1000:
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c


In [57]:
a = torch.randn(size=(), requires_grad=True)
a


tensor(0.4857, requires_grad=True)

In [58]:
d = f(a)
d

tensor(1989.2413, grad_fn=<MulBackward0>)

In [59]:
d.backward()


In [60]:
a.grad, a.grad == d / a


(tensor(4096.), tensor(True))

## Exercises


In [61]:
"""
This error occurs because by default, PyTorch's automatic differentiation framework assumes that the result of a computation is a scalar value. When calling backward(), PyTorch expects a scalar tensor to propagate gradients through the computational graph properly.

To overcome this issue and compute gradients for tensors with multiple elements, you need to specify a scalar quantity for backpropagation. This could involve taking the sum or mean of the tensor d before calling backward().
"""
a = torch.randn(size=(3, ), requires_grad=True)
d = f(a)
scalar_d = d.sum()  # Taking the sum to obtain a scalar quantity
scalar_d.backward()
print(d, scalar_d, a, a.grad)



tensor([1418.8359,  741.6139,  -53.5094], grad_fn=<MulBackward0>) tensor(2106.9402, grad_fn=<SumBackward0>) tensor([ 0.6928,  0.3621, -0.0261], requires_grad=True) tensor([2048., 2048., 2048.])
